In [112]:
import pandas as pd
from datetime import datetime
import re

In [113]:
def open_file(filename):
    try:
        f = open(filename, "r")
        text = f.read()
    except:
        f = open(filename, "r",encoding='latin-1')
        text = f.read()
        # encoding='latin-1'
    lines = text.split('\n')
    return lines


def clean_name(name: str) -> str:
    char_add_ons = ['V.O.', "'S COM VOICE",
                    'Cont\'d', 'O.S.', "'S", "VOICE", "INTERCOM"]
    for txt in char_add_ons:
        if txt in name:
            name = name.replace(txt, '')
    return re.sub("[\(\[].*?[\)\]]", "", name.strip())


def get_title(lines):
    series_ind = 0
    for i in range(len(lines)):
        if "STAR TREK: THE NEXT GENERATION" in lines[i] or 'STAR TREK: DEEP SPACE NINE' in lines[i]:
            series_ind = i
            break
    return str(lines[series_ind+2]).strip().replace('"','')


def get_date(lines):
    draft_ind = 0
    for i in range(len(lines)):
        if "FINAL DRAFT" in lines[i]:
            series_ind = i
            break
    try:
        date = str(lines[series_ind+2])
        date = date.strip()
        final = datetime.strptime(date, '%B %d, %Y')
    except:
        print(lines[series_ind+2])
        final = "AHHH"
    return final



In [114]:
def get_quotes(lines):
#    characters = [x for x in lines if '\t\t\t\t\t' in x]
    char_index = [i for i in range(len(lines)) if '\t\t\t\t\t' in lines[i]]
    scenes_index = [j for j in range(len(lines)) if 'INT.' in lines[j] or 'EXT.' in lines[j]]

    data = []

    for i in char_index:
        name = lines[i].replace('\t\t\t\t\t', '')
        if "FADE OUT" in name:
            continue

        j = i+1
        next_line = lines[j]

        quote = ""

        while next_line != '':
            next_line = next_line.replace('\t', ' ')
            quote += re.sub("[\(\[].*?[\)\]]", "", next_line)
            j = j+1
            next_line = lines[j]

        if quote!="":
            print(quote,lines[j],lines[j-1])
            print(j)
            print(scenes_index)
            scene = lines[max([x for x in scenes_index if x < i])]

            data.append([name, quote,scene])

    test_df = pd.DataFrame(data, columns=['character', 'quote','scene'])
    return (test_df)

def clean_location(loc:str):
    views = ['INT.','EXT.']
    output = loc
    for v in views:
        if v in output:
            output = output[output.index(v)+5:]
    if '(' in output:
        output = output[:output.index('(')-1]
    elif '-' in output:
        output = output[:output.index('-')-1]
    return(output)

def get_view(loc:str):
    output = None
    views = ['INT.','EXT.']
    for v in views:
        if v in loc:
            output=v
    return(output)


def create_df(filepath: str) -> pd.DataFrame:
    lines = open_file(filepath)
    test_df = get_quotes(lines)
    test_df['location']=test_df['scene'].apply(clean_location)
    test_df['view']=test_df['scene'].apply(get_view)
    test_df['character'] = test_df['character'].apply(clean_name)
    test_df['episode'] = get_title(lines)
    test_df['date']=get_date(lines)
    return test_df


In [115]:
x=create_df('scripts_ds9/402.txt')
x

   You will be assimilated.   Resistance is futile.  			Resistance is futile.
182
[193, 222, 230, 241, 245, 257, 263, 267, 331, 376, 491, 526, 616, 639, 654, 704, 799, 900, 982, 1126, 1139, 1285, 1289, 1442, 1444, 1572, 1682, 1692, 1821, 1892, 2091, 2185, 2189, 2237, 2245, 2312, 2328, 2419, 2489, 2574, 2620, 2624, 2635, 2660, 2735, 2750, 2777, 2818, 2823, 2931, 2936, 3034, 3086, 3088, 3118, 3132, 3144, 3149, 3174, 3185, 3193, 3197, 3235, 3249, 3254, 3277, 3300, 3308, 3327, 3331, 3343, 3362, 3367, 3391, 3396, 3472, 3480, 3502, 3506, 3577, 3584, 3589, 3596, 3749, 3753, 3863, 3885, 3915, 3968, 3972, 4101, 4154, 4173, 4183, 4189, 4199, 4236, 4264, 4275, 4313, 4345, 4349, 4366, 4371, 4411, 4421, 4515, 4520, 4535, 4542, 4639, 4799, 4865, 4873, 4882, 4894, 4907, 4912, 4922, 4929, 4943, 4947, 4967, 5007, 5020, 5041, 5058, 5080, 5111, 5156, 5265, 5281, 5436, 5438, 5504, 5701, 5705, 5747, 5755, 5810, 5814, 5864, 5899, 5904, 5910, 5927, 5933, 5948, 5954, 5961, 5987, 5997, 6018, 6023, 6058, 6106, 

ValueError: max() arg is an empty sequence

In [ ]:
lines = open_file('scripts_ds9/408.txt')
get_date(lines)

datetime.datetime(1992, 11, 17, 0, 0)

In [ ]:
from os import listdir
from os.path import isfile, join
mypath = 'scripts_ds9/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles

['504.txt',
 '510.txt',
 '538.txt',
 '470.txt',
 '464.txt',
 '458.txt',
 '459.txt',
 '465.txt',
 '471.txt',
 '539.txt',
 '511.txt',
 '505.txt',
 '513.txt',
 '507.txt',
 '498.txt',
 '467.txt',
 '472.txt',
 '466.txt',
 '499.txt',
 '506.txt',
 '512.txt',
 '516.txt',
 '502.txt',
 '489.txt',
 '462.txt',
 '476.txt',
 '477.txt',
 '463.txt',
 '488.txt',
 '503.txt',
 '517.txt',
 '529.txt',
 '501.txt',
 '515.txt',
 '449.txt',
 '475.txt',
 '461.txt',
 '460.txt',
 '474.txt',
 '448.txt',
 '514.txt',
 '500.txt',
 '528.txt',
 '567.txt',
 '573.txt',
 '413.txt',
 '407.txt',
 '406.txt',
 '412.txt',
 '572.txt',
 '566.txt',
 '570.txt',
 '564.txt',
 '558.txt',
 '404.txt',
 '410.txt',
 '438.txt',
 '439.txt',
 '411.txt',
 '405.txt',
 '559.txt',
 '565.txt',
 '571.txt',
 '549.txt',
 '575.txt',
 '561.txt',
 '429.txt',
 '415.txt',
 '414.txt',
 '428.txt',
 '560.txt',
 '574.txt',
 '548.txt',
 '562.txt',
 '416.txt',
 '402.txt',
 '403.txt',
 '417.txt',
 '563.txt',
 '546.txt',
 '552.txt',
 '432.txt',
 '426.txt',
 '42